[View in Colaboratory](https://colab.research.google.com/github/divsinha99/Black_Friday/blob/master/BlackFriday_Pipeline.ipynb)

In [1]:
from google.colab import files
uploaded = files.upload()

Saving test_HujdGe7.zip to test_HujdGe7.zip
Saving train_oSwQCTC.zip to train_oSwQCTC.zip


In [0]:
import pandas as pd
#path = "D:\Data_science\practice\AV_Black_Friday"

def load_data():
  #  csv_train_path = os.path.join(path, "train.csv")
  #  csv_test_path = os.path.join(path, "test.csv")
    return pd.read_csv("train_oSwQCTC.zip",compression='zip', sep=','), pd.read_csv("test_HujdGe7.zip",compression='zip', sep=',')

In [3]:
train_df, test_df = load_data()
train_df.head(2)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200


In [5]:
test_df.head(2)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN


# Imputing Missing values in Prod Category

In [0]:
# Missing value imputation
train_df['Product_Category_2'].fillna(0.000000, inplace = True)
train_df['Product_Category_3'].fillna(0.000000, inplace = True)

In [0]:
purchase_df = train_df.drop("Purchase", axis=1) # drop labels for training set
purchase_labels = train_df["Purchase"].copy()

In [0]:

from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [0]:
import numpy as np

In [11]:
list(purchase_df.select_dtypes(include=[np.number]).columns)[3:]

['Product_Category_1', 'Product_Category_2', 'Product_Category_3']

In [12]:
list(purchase_df.columns)[2:8]

['Gender',
 'Age',
 'Occupation',
 'City_Category',
 'Stay_In_Current_City_Years',
 'Marital_Status']

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin


class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [14]:
!pip install sklearn_pandas

In [0]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn_pandas import DataFrameMapper

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelBinarizer

num_attribs = list(purchase_df.select_dtypes(include=[np.number]).columns)[3:]
#cat_attribs = list(purchase_df.select_dtypes(exclude=[np.number]).columns) + \
#                       ["User_ID","Occupation","Marital_Status"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ("interaction_features", PolynomialFeatures(degree=2, interaction_only=False,  
                        include_bias=False)),
        ('std_scaler', StandardScaler()),
    ])

dummies = list(purchase_df.columns)[2:8]

mapper = DataFrameMapper(
    [(d, LabelBinarizer()) for d in dummies]
)


cat_pipeline = Pipeline([
        ("binarizer_mapper", mapper),
    ])

In [0]:
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [18]:
purchase_prepared = full_pipeline.fit_transform(purchase_df)
purchase_prepared

array([[-0.61080871, -1.08365403, -0.61464112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.11891197, -0.11832323,  1.62510562, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.67565594, -1.08365403, -0.61464112, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 3.70806897, -1.08365403, -0.61464112, ...,  0.        ,
         1.        ,  1.        ],
       [ 3.70806897, -1.08365403, -0.61464112, ...,  0.        ,
         0.        ,  0.        ],
       [ 3.70806897, -1.08365403, -0.61464112, ...,  0.        ,
         1.        ,  1.        ]])

In [19]:
purchase_prepared.shape

(550068, 47)

# Linear Regression

In [20]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(purchase_prepared, purchase_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
# let's try the full pipeline on a few training instances
some_data = train_df.iloc[:5]
some_labels = purchase_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))

Predictions: [ 8518. 12202.  5492.  5718.  7454.]


In [22]:
print("Labels:", list(some_labels))

Labels: [8370, 15200, 1422, 1057, 7969]


In [23]:
some_data_prepared

array([[-0.61080871, -1.08365403, -0.61464112, -0.56882992, -0.68879926,
        -0.39145861, -0.84857881, -0.46981945, -0.55973351,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ],
       [-1.11891197, -0.11832323,  1.62510562, -0.69629929, -0.56983383,
         0.09451591, -0.48489511,  0.86533373,  1.47823793,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.     

In [24]:
from sklearn.metrics import mean_squared_error

purchase_predictions = lin_reg.predict(purchase_prepared)
lin_mse = mean_squared_error(purchase_labels, purchase_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

4416.764034328058

In [25]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(purchase_labels, purchase_predictions)
lin_mae

3317.2081015438093

# Decision Tree regressor

In [26]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(purchase_prepared, purchase_labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=42, splitter='best')

In [27]:
purchase_predictions = tree_reg.predict(purchase_prepared)
tree_mse = mean_squared_error(purchase_labels, purchase_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

2290.8062646412236

In [28]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(tree_reg, purchase_prepared, purchase_labels, scoring="mean_squared_error", cv=10)
print(scores)

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scorin

[-10865711.55337592 -11055561.34522807 -10970309.26980346
 -10745609.6619496  -10968876.90719366 -10848167.87684952
 -10855602.99438703 -10884565.59119491 -10947364.76114509
 -10822996.0681491 ]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


In [29]:
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([3296.31787808, 3324.99042784, 3312.14571989, 3278.0496735 ,
       3311.92948403, 3293.65570102, 3294.7842106 , 3299.17650198,
       3308.6802144 , 3289.83222492])

In [30]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [3296.31787808 3324.99042784 3312.14571989 3278.0496735  3311.92948403
 3293.65570102 3294.7842106  3299.17650198 3308.6802144  3289.83222492]
Mean: 3300.956203626166
Standard deviation: 12.83529000705941


In [31]:

lin_scores = cross_val_score(lin_reg, purchase_prepared, purchase_labels,scoring="mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/m

Scores: [4306.22924972 4359.66908248 4355.39121959 4311.57733066 4352.25164658
 4357.34893336 4331.29848363 4363.9884015  4368.90586934 6169.96984979]
Mean: 4527.6630066632115
Standard deviation: 547.8252898501063


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


# Random Forest Regressor

In [32]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(purchase_prepared, purchase_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [35]:
purchase_predictions = forest_reg.predict(purchase_prepared)
forest_mse = mean_squared_error(purchase_labels, purchase_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

2377.216668321207

In [36]:
forest_scores = cross_val_score(forest_reg, purchase_prepared, purchase_labels,scoring="mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/usr/local/lib/python3.6/dist-packages/sklearn/m

Scores: [3094.05214576 3088.59007647 3078.17125885 3089.61999872 3091.006107  ]
Mean: 3088.2879173593224
Standard deviation: 5.381958827852208
